In [ ]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset

your_dataset = pd.read_csv('news.csv')
your_dataset = your_dataset.fillna(0)
your_dataset['News'] = your_dataset['News'].astype(str)
your_dataset_nlp = Dataset.from_pandas(your_dataset)

model_name = "chrisliu298/mmlu-economics_classifier"
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

def tokenize_function(examples):
    return tokenizer(examples["News"], padding="max_length", truncation=True)


tokenized_inputs = your_dataset_nlp.map(tokenize_function, batched=True)

model.eval()
with torch.no_grad():
    input_ids = torch.tensor(tokenized_inputs['input_ids']).to(device)
    attention_mask = torch.tensor(tokenized_inputs['attention_mask']).to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

predictions = torch.argmax(logits, dim=-1)
your_dataset['label'] = predictions.cpu().numpy()

your_dataset.to_csv('news_with_labels.csv', index=False)

print("Размеченный датасет сохранён в 'news_with_labels.csv'")

with open('model.pkl', 'wb') as f:
    pickle.dump((model, tokenizer), f)

print("Модель и токенизатор успешно сохранены в model.pkl.")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/396 [00:00<?, ? examples/s]

Размеченный датасет сохранён в 'news_with_labels.csv'
Модель и токенизатор успешно сохранены в model.pkl.
